In [61]:
!pip3 install qrcode
!pip3 install python-resize-image


You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [23]:
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


sheet_id = "1fTaZGNrpIC6ObWOA46TQ_eA6kR5E3hh36-CBIv0EKCY"
eunyu_gid = 0
matt_gid = 865169686

url_eunyu = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid={eunyu_gid}&format=csv"
url_matt = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?gid={matt_gid}&format=csv"

eunyu_df = pd.read_csv(url_eunyu)
matt_df = pd.read_csv(url_matt)

In [24]:
total_names = eunyu_df[['Name', 'Connection']][:-1].append(matt_df[['Name', 'Connection']][:-4])
total_names.fillna('Others', inplace=True)
total_names

/var/folders/kc/83g7q_n10f352lr0d32t57lw0000gn/T/ipykernel_86192/2889858979.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total_names = eunyu_df[['Name', 'Connection']][:-1].append(matt_df[['Name', 'Connection']][:-4])


,Name,Connection
0,Bong Xin Yi,Hall friends
1,Yuki,Hall friends
2,Dion,Hall friends
3,Giwen,Hall friends
4,Jon San,Hall friends
...,...,...
152,Xue Min,Others
153,Yanzhang,Others
154,Gin,Others
155,Hafiz,Others


In [25]:
def convert_to_snake_case(name, connection):
    # Convert the string to lowercase
    lowercase_string = name.lower()
    connection_string = connection.lower()

    # Replace spaces with hyphens
    hyphenated_string = f'{lowercase_string}-{connection_string}'.replace(' ', '-')

    # Convert the hyphenated string to snake case
    snake_case_string = hyphenated_string.replace(' ', '_')

    

    return snake_case_string


base_url = 'https://mattyubygrace.com/feed?userId='
url = []
for idx, row in total_names.iterrows():
    url.append(f"{base_url}{convert_to_snake_case(row['Name'], row['Connection'])}")

total_names['url'] = url

total_names.to_clipboard()


In [76]:
# Generate QR codes

import qrcode

qr_codes = []

def make_qr_code(link):
  qr = qrcode.QRCode(
      version=8,
      error_correction=qrcode.constants.ERROR_CORRECT_L,
      box_size=6,
      border=0,
  )
  qr.add_data(link)
  qr.make(fit=True)
  
  return qr.make_image(fill_color="rgb(198, 139, 87)", back_color="white")

for idx, row in total_names.iterrows():
  img = make_qr_code(row['url'])
  qr_codes.append(img)


In [83]:
from PIL import Image, ImageDraw, ImageFont

cormorant_font = './template/Cormorant-VariableFont_wght.ttf'

def create_front(name):
    front = Image.open('./template/front.png')
    canvas = ImageDraw.Draw(front)
    canvas.text((840, 430), name, font=ImageFont.truetype(
        cormorant_font, 200), fill=(243, 235, 225), anchor="mm")
    return front

def create_back(qr_code, link):
    back = Image.open('./template/back.png')
    canvas = ImageDraw.Draw(back)
    canvas.text((440, 650), 'https://mattyubygrace.com', font=ImageFont.truetype(cormorant_font, 50),
                fill=(243, 235, 225), anchor="mm")
    back.paste(qr_code, (300, 300))
    return back


for idx, row in total_names[:1].iterrows():
    front = create_front(row['Name'])
    back = create_back(qr_codes[idx], row['url'])

    base_name = f"{row['Name']}"

    front.save(f"output/{base_name}-front.png")
    back.save(f"output/{base_name}-back.png")
